# Rocket Game
> This notebook provides explanations and examples of various components that are incorporated in their project.

- toc: true
- categories: [python]

## Frontend Code

In [ ]:
<html>
<head>
  <meta charset="UTF-8">
  <title>Rocket Launch Simulator</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 0;
    }
    h1 {
      font-size: 36px;
      text-align: center;
    }
    #game-container {
      display: flex;
      flex-direction: column;
      align-items: center;
      margin-top: 40px;
    }
        #game-form {
      display: flex;
      flex-direction: column;
      align-items: center;
      margin-bottom: 20px;
    }
    label {
      font-size: 20px;
      margin-right: 10px;
    }
    input {
      font-size: 20px;
      margin-bottom: 10px;
      padding: 5px;
    }
    button {
      font-size: 20px;
      padding: 10px 20px;
      background-color: #4CAF50;
      color: white;
      border: none;
      cursor: pointer;
      margin-top: 10px;
    }
    button:hover {
      background-color: #3e8e41;
    }
    #result-container {
      display: none;
      text-align: center;
    }
    #canvas-container {
      margin-top: 20px;
    }
    #success-animation,
    #failure-animation {
      display: none;
      text-align: center;
    }
    .success,
    .failure {
      font-size: 24px;
    }
     @keyframes fadeIn {
      0% {
        opacity: 0;
      }
      100% {
        opacity: 1;
      }
    }
    @keyframes slideUp {
      0% {
        transform: translateY(200px);
        opacity: 0;
      }
      100% {
        transform: translateY(0);
        opacity: 1;
      }
    }
</style>
</head>
<body>
  <h1>Rocket Launch Simulator</h1>
  <div id="game-container">
    <form id="game-form">
      <label for="thrust">Thrust:</label>
      <input type="number" id="thrust" name="thrust" required>
      <label for="drag">Drag:</label>
      <input type="number" id="drag" name="drag" required>
      <label for="time">Time:</label>
      <input type="number" id="time" name="time" required>
      <button type="submit">Launch Rocket</button>
    </form>
    <div id="result-container">
      <h2>Result:</h2>
      <p id="velocity"></p>
      <p id="altitude"></p>
      <div id="canvas-container">
        <canvas id="canvas" width="600" height="600"></canvas>
      </div>
      <div id="success-animation">
        <p class="success">Success! The rocket reached outer space.</p>
      </div>
      <div id="failure-animation">
        <p class="failure">Failure! The rocket did not reach outer space.</p>
      </div>
    </div>
  </div>

  <script>
    const form = document.getElementById('game-form');
    const resultContainer = document.getElementById('result-container');
    const velocityElement = document.getElementById('velocity');
    const altitudeElement = document.getElementById('altitude');
    const successAnimation = document.getElementById('success-animation');
    const failureAnimation = document.getElementById('failure-animation');
    const canvas = document.getElementById("canvas");
    const ctx = canvas.getContext("2d");

    const spaceGradient = ctx.createLinearGradient(0, 0, 0, canvas.height);
    spaceGradient.addColorStop(0, 'black');
    spaceGradient.addColorStop(1, 'navy');

    // Wait for the background to load before starting the animation
    window.addEventListener('load', function() {
    // Draw background
    ctx.fillStyle = spaceGradient;
    ctx.fillRect(0, 0, canvas.width, canvas.height);

    // Start animation
    rocketImages.success1.onload = function() {
        drawRocket(380, rocketImages.success1);
    };
    rocketImages.success2.onload = function() {
        // Don't draw the rocket here since it's used in animateRocket()
        // drawRocket(380, rocketImages.success2);
    };
    rocketImages.failure.onload = function() {
        // Don't draw the rocket here since it's used in animateRocket()
        // drawRocket(380, rocketImages.failure);
    };
    });

    form.addEventListener('submit', (e) => {
      e.preventDefault();

      const thrust = parseFloat(document.getElementById('thrust').value);
      const drag = parseFloat(document.getElementById('drag').value);
      const time = parseFloat(document.getElementById('time').value);

      const data = {
        thrust: thrust,
        drag: drag,
        time: time
      };

      // Send a POST request to the Flask API
      fetch('https://ctrpe.duckdns.org/api/rocket/game', {
          method: 'POST',
          mode: 'cors',
          headers: {
            'Content-Type': 'application/json'
          },
          body: JSON.stringify(data)
        })
        .then(response => response.json())
        .then(result => {
          // Update the game interface with the result
          velocityElement.textContent = `Velocity: ${result.velocity} m/s`;
          altitudeElement.textContent = `Altitude: ${result.altitude} m`;
          resultContainer.style.display = 'block';

          // Show success or failure animation based on the altitude
          if (result.altitude >= 100000) {
            successAnimation.style.display = 'block';
            successAnimation.style.animationName = 'fadeIn';
            successAnimation.style.opacity = 1;
            successAnimation.style.animationDuration = '4s';
            successAnimation.style.animationFillMode = 'forwards';
            successAnimation.style.animationTimingFunction = 'ease-in-out';

            // Start success animation
            animateRocket(380, 'success');
          } else {
            failureAnimation.style.display = 'block';
            failureAnimation.style.animationName = 'slideUp';
            failureAnimation.style.transform = 'translateY(0)';
            failureAnimation.style.opacity = 1;
            failureAnimation.style.animationDuration = '2s';
            failureAnimation.style.animationFillMode = 'forwards';
            failureAnimation.style.animationTimingFunction = 'ease-in-out';

            // Start failure animation
            animateRocket(380, 'failure');
          }
        })
        .catch(error => console.error('Error:', error));
    });

    let rocketImages = {
      success1: new Image(),
      success2: new Image(),
      failure: new Image()
    };
    rocketImages.success1.src = 'rocket-success1.png';
    rocketImages.success2.src = 'rocket-success2.png';
    rocketImages.failure.src = 'rocket-failure.png';

    document.addEventListener('DOMContentLoaded', function() {
      rocketImages.success1.onload = function() {
        drawRocket(380, rocketImages.success1);
      };
      rocketImages.success2.onload = function() {
        // Don't draw the rocket here since it's used in animateRocket()
        // drawRocket(380, rocketImages.success2);
      };
      rocketImages.failure.onload = function() {
        // Don't draw the rocket here since it's used in animateRocket()
        // drawRocket(380, rocketImages.failure);
      };
    });

    function drawRocket(yPos, rocketImage) {
      ctx.clearRect(0, 0, canvas.width, canvas.height);
      ctx.drawImage(rocketImage, 180, yPos, 40, 80);
    }

    function animateRocket(yPos, animationType) {
      let frame = 0;
      let rocketImage;
      let smokeImage = new Image();
      smokeImage.src = "smoke.png";

      if (animationType === 'success') {
        rocketImage = rocketImages.success1;
      } else {
        rocketImage = rocketImages.failure;
      }

      function animateOneFrame() {
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        drawRocket(yPos, rocketImage);

        // Draw the smoke trail for the failure animation
        if (animationType === 'failure') {
          let smokeX = 195;
          ctx.drawImage(smokeImage, smokeX, yPos + 80, 20, 20);
          smokeX += Math.random() * 20 - 10;
        }

        frame++;
        yPos -= 2;

        if (animationType === 'success') {
          if (frame < 60) {
            rocketImage = rocketImages.success1;
          } 
          else if (frame < 90) {
            rocketImage = rocketImages.success2;
            }  
          else {
            console.log("Animation completed");
          }
        }

        if (frame < 120) {
          window.requestAnimationFrame(animateOneFrame);
        } else {
          console.log("Animation completed");
        }
      }

      animateOneFrame();
    }
  </script>
</body>

## Backend Code

In [ ]:
from flask import Flask, jsonify, request, Blueprint, app
from flask_restful import Resource, Api

rocket_api = Blueprint('rocket_api', __name__, 
                            url_prefix='/api/rocket')
api = Api(rocket_api)

# Constants
GRAVITY = 9.8  # Acceleration due to gravity

# Create a resource for handling game requests
class GameResource(Resource):
    def post(self):
        # Get the player's input from the request
        data = request.get_json()

        # Process the player's input and calculate the rocket's trajectory
        thrust = data['thrust']
        drag = data['drag']
        time = data['time']

        velocity = thrust - drag - GRAVITY * time
        altitude = 0.5 * (thrust - drag) * time**2 - GRAVITY * time**2

        # Return the result as a JSON response
        return jsonify(velocity=velocity, altitude=altitude)

# Add the resource to the API
api.add_resource(GameResource, '/game')

## Frontend Step by Step

1. The HTML structure starts with the opening <body> tag.
2. The h1 tag displays the heading "Rocket Launch Simulator".
3. Inside the <body>, there is a <div> element with the id="game-container". This container wraps the entire game interface.
4. Inside the game container, there is a <form> element with the id="game-form". This form contains input fields for the user to provide thrust, drag, and time values for the rocket launch.
5. Each input field has a corresponding <label> element for better accessibility.
6. The form also includes a <button> with the "Launch Rocket" text for submitting the form.
7. After the form, there is a <div> element with the id="result-container". This container holds the result of the rocket launch simulation.
8. Inside the result container, there are two <p> elements with the id="velocity" and id="altitude". These elements will display the calculated velocity and altitude values.
9. A nested div with the id="canvas-container" wraps a canvas element with the id="canvas". This canvas will be used to draw the animation.
10. Following the canvas, there are two additional <div> elements with the id="success-animation" and id="failure-animation". These divs contain paragraphs that will display success or failure messages during the animation.
11. The JavaScript code starts with the opening script tag.
12. Several constant variables are declared using const keyword to store references to various elements in the HTML structure.
13. The spaceGradient variable creates a linear gradient using the ctx.createLinearGradient() method to set the canvas background.
14. A load event listener is added to the window object to wait for the background to load before starting the animation.
15. Inside the event listener, the background is drawn on the canvas using ctx.fillRect() and ctx.fillStyle properties.
16. The rocketImages object is declared to store instances of the Image class for different rocket images.
17. The image sources are assigned to the respective src properties of the rocketImages object.
18. Another event listener is added to the DOMContentLoaded event to ensure that the rocket images are loaded before drawing the rocket on the canvas.
19. The drawRocket() function is defined to draw the rocket on the canvas at the specified yPos with the given rocketImage.
20. The animateRocket() function is defined to animate the rocket by repeatedly redrawing it on the canvas at different yPos positions.
21. Inside the animateRocket() function, a frame variable is initialized to track the animation frames and determine which rocket image to display.
22. A smokeImage variable is declared and assigned the source image for the smoke trail.
23. The animation starts by calling the animateOneFrame() function.
24. The animateOneFrame() function clears the canvas, draws the rocket, and updates the frame and yPos variables.
25. If the animation is a success, different rocket images are displayed at different frames.
26. For the failure animation, a smoke trail is drawn by randomly adjusting the smokeX position.
27. The animation is continued by calling window.requestAnimationFrame(animateOneFrame) until the specified number of frames is reached.
28. The final <script> tag is closed.
29. The HTML structure is closed with the closing </body> tag.

### Backend Step by Step

1. Import the necessary modules from the Flask and Flask-RESTful libraries. These modules are required to create a RESTful API in Flask.
2. Create a Flask blueprint named "rocket_api" using the Blueprint() function. The blueprint will define a set of routes for the rocket API.
3. Specify the URL prefix for the rocket API as "/api/rocket" by passing the url_prefix parameter to the Blueprint() function.
4. Create an instance of the Api class from Flask-RESTful and name it "api". This instance will handle the routing and resource management for the API.
5. Define a constant variable named "GRAVITY" and assign it a value of 9.8. This variable represents the acceleration due to gravity.
6. Create a class named "GameResource" that inherits from the Resource class provided by Flask-RESTful. This class will handle the game requests for the rocket API.
7. Inside the "GameResource" class, define a "post" method. This method will be called when a POST request is made to the game resource.
8. Within the "post" method, extract the player's input from the request by calling the request.get_json() method. The input data is expected to be in JSON format.
9. Process the player's input by retrieving the "thrust", "drag", and "time" values from the input data. Perform calculations to determine the rocket's velocity and altitude based on these values.
10. Return the result as a JSON response using the jsonify() function from Flask. The response will contain the calculated velocity and altitude values.